In [2]:
import odrive
from odrive.enums import *
import time
import math
import numpy as np
import sympy as sp
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt

## Connect to Odrive

In [3]:
odrv1 = odrive.find_any(serial_number = "20773394304B" )
if odrv1 is not None:
    print('Connected!\nODrive1 serial {}'.format(odrv1.serial_number))
else:
    print('Not connected')
    
odrv2 = odrive.find_any(serial_number = "337231563536")
if odrv2 is not None:
    print('Connected!\nODrive2 serial {}'.format(odrv2.serial_number))
else:
    print('Not connected')

m0  = odrv1.axis0
m1  = odrv1.axis1
m00 = odrv2.axis0
m11 = odrv2.axis1

Connected!
ODrive1 serial 35696338546763
Connected!
ODrive2 serial 56565547021622


## Setup

In [4]:
def SetParameter(odrv0):
    odrv0.axis0.motor.config.current_lim     = 5 #A
    odrv0.axis1.motor.config.current_lim     = 5 #A
    odrv0.axis0.controller.config.vel_limit  = 1000000
    odrv0.axis1.controller.config.vel_limit  = 1000000
    return()

def CheckMotorError(odrv0):
    axis0error = odrv0.axis0.error 
    axis1error = odrv0.axis1.error 
    axis0motorerror = odrv0.axis0.motor.error 
    axis1motorerror = odrv0.axis1.motor.error 
    axis0controllererror = odrv0.axis0.controller.error 
    axis1controllererror = odrv0.axis0.controller.error 
    print("error:          ", axis0error, axis1error)
    print("motorerror:     ", axis0motorerror, axis1motorerror)
    print("controllererror:", axis0controllererror, axis1controllererror)
    return()

def MotorClearError(odrv0):
    odrv0.axis0.error = 0
    odrv0.axis1.error = 0
    odrv0.axis0.motor.error = 0
    odrv0.axis1.motor.error = 0
    odrv0.axis0.controller.error = 0
    odrv0.axis1.controller.error = 0
    odrv0.axis0.encoder.error = 0
    odrv0.axis1.encoder.error = 0
    return()

In [5]:
SetParameter(odrv1)
SetParameter(odrv2)
print("Parameter Set!")

Parameter Set!


In [6]:
print("odrive1")
CheckMotorError(odrv1)
print("odrive2")
CheckMotorError(odrv2)

MotorClearError(odrv1)
MotorClearError(odrv2)
print("Error Clear!")

odrive1
error:           0 0
motorerror:      0 0
controllererror: 0 0
odrive2
error:           0 0
motorerror:      0 0
controllererror: 0 0
Error Clear!


## Motor Commands

In [7]:
def Calibration(odrv0):
    odrv0.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    print('calibration done')

def ClosedLoopControlMode(odrv0):
    odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
    odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

def IdleMode(odrv0):
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

def ControllerPositionControl(odrv0): # default mode
    odrv0.axis0.controller.config.control_mode = CTRL_MODE_POSITION_CONTROL
    odrv0.axis1.controller.config.control_mode = CTRL_MODE_POSITION_CONTROL
    print("Current Control Mode :{}".format(odrv0.axis0.controller.config.control_mode))

def ControllerCurrentControl(odrv0):
    odrv0.axis0.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL
    odrv0.axis1.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL
    print("Control Mode :{}".format(odrv0.axis0.controller.config.control_mode))

def MaxCurrentProtection(current, current_limit = 10):
    current  = min(np.abs(current), current_limit) * np.sign(current) 
    return(current)

In [8]:
Calibration(odrv1)
Calibration(odrv2)

calibration done
calibration done


In [11]:
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

In [10]:
IdleMode(odrv1)
IdleMode(odrv2)

In [12]:
ControllerCurrentControl(odrv1)
ControllerCurrentControl(odrv2)

Control Mode :1
Control Mode :1


# Forward Kinematics & Jacobian (Joint space -- Foot space)

In [13]:
(thetaL_sym, thetaR_sym) = sp.symbols("""thetaL_sym thetaR_sym """, real = True)
(x_cartesian, y_cartesian) = sp.symbols("""x_cartesian y_cartesian """, real = True)

def T(theta, x, y):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])

l1 = 0.09
l2 = 0.16
w = 0.07

x_r = -w/2 + l1*sp.cos(thetaR_sym)
x_l = w/2 + l1*sp.cos(thetaL_sym)

y_r = l1*sp.sin(thetaR_sym)
y_l = l1*sp.sin(thetaL_sym)

theta3_sym = sp.atan2(y_l - y_r, x_l - x_r)
L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

FK = T(thetaL_sym, w/2, 0)@T((sp.pi - thetaL_sym + theta3_sym), l1, 0)@sp.Matrix([L/2, -sp.sqrt(l2**2 - (L/2)**2), 1])
FK = FK[:2,:]
FK.simplify()
FK_fast = lambdify((thetaR_sym, thetaL_sym), FK) # (x,y) = FK_fast(thetaR,thetaL)

J = FK.jacobian([thetaR_sym, thetaL_sym]).evalf()
J_fast = lambdify((thetaR_sym, thetaL_sym), J) # J_fast(thetaR,thetaL)

# Define zero angle (Joint space)

In [14]:
#front foot
m0_home_pos = m0.encoder.pos_estimate
m1_home_pos = m1.encoder.pos_estimate
theta_R_home = -((m0_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_home = -((m1_home_pos) / 8192 * 2 * np.pi)
# theta_R_home_d = theta_R_home / (2 * np.pi) * 360
# theta_L_home_d = theta_L_home / (2 * np.pi) * 360
print(m0_home_pos)
print(m1_home_pos)

-96.078125
4749.234375


In [15]:
#back foot
m00_home_pos = m00.encoder.pos_estimate
m11_home_pos = m11.encoder.pos_estimate
theta_R_home = -((m00_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_home = -((m11_home_pos) / 8192 * 2 * np.pi)
# theta_R_home_d = theta_R_home / (2 * np.pi) * 360
# theta_L_home_d = theta_L_home / (2 * np.pi) * 360
print(m00_home_pos)
print(m11_home_pos)

-2431.25
155.234375


In [ ]:
# # Without encoder read position
# m0_home_pos = -1157.032470703125
# m1_home_pos = 707.75048828125
# theta_R_home = -((m0_home_pos) / 8192 * 2 * np.pi) + np.pi
# theta_L_home = -((m1_home_pos) / 8192 * 2 * np.pi)

# Define tiptoe home location (Joint space)

In [16]:
#Front foot home location
m0_equilibrium_toe_pos = m0.encoder.pos_estimate
m1_equilibrium_toe_pos = m1.encoder.pos_estimate
theta_R_eq = - ((m0_equilibrium_toe_pos - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_eq = - ((m1_equilibrium_toe_pos - m1_home_pos) / 8192 * 2 * np.pi)
theta_R_eq_d = theta_R_eq / (2 * np.pi) * 360
theta_L_eq_d = theta_L_eq / (2 * np.pi) * 360
print("Odrv1 \nR:", theta_R_eq_d, "\nL:", theta_L_eq_d)

Odrv1 
R: 140.50071716308594 
L: 35.52909851074218


In [17]:
#Back foot home location
m00_equilibrium_toe_pos = m00.encoder.pos_estimate
m11_equilibrium_toe_pos = m11.encoder.pos_estimate
theta_R_eq = - ((m00_equilibrium_toe_pos - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_eq = - ((m11_equilibrium_toe_pos - m11_home_pos) / 8192 * 2 * np.pi)
theta_R_eq_d = theta_R_eq / (2 * np.pi) * 360
theta_L_eq_d = theta_L_eq / (2 * np.pi) * 360
print("Odrv2 \nR:", theta_R_eq_d, "\nL:", theta_L_eq_d)

Odrv2 
R: 143.6572265625 
L: 42.42851257324219


# Record trajectory (Joint space)

In [ ]:
# theta_R_eq_array = []
# theta_L_eq_array = []

# t0 = 0
# t1 = 0
# t_start = time.time()
# duration = 10
# count = 0

# while (time.time() - t_start < duration):
    
#     t0 = time.time()
#     #read the encoder position & velocity
#     m0_encoder_pos = m0.encoder.pos_estimate
#     m1_encoder_pos = m1.encoder.pos_estimate
#     #current angle(rad) & velocity(rad/s)
#     theta_R_eq = - ((m0_encoder_pos - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
#     theta_L_eq = - ((m1_encoder_pos - m1_home_pos) / 8192 * 2 * np.pi)
#     #current angle(degree)
# #     theta_R_eq_d = theta_R_eq / (2 * np.pi) * 360
# #     theta_L_eq_d = theta_L_eq / (2 * np.pi) * 360
    
#     theta_R_eq_array.append(theta_R_eq)
#     theta_L_eq_array.append(theta_L_eq)
    
#     count += 1

#     t1 = time.time()
#     time_spent = t1 - t0
# #     print(time_spent)
# #     time.sleep(0.005 - time_spent)
    
# print(count,'iterations in', duration, 'second')
# print(count/duration, 'Hz')

# Sine Trajectory (Foot Space)

In [ ]:
# DownAMP = 0.01
# UpperAMP = 0.05
# StanceHeight = 0.18
# StepLength = 0.12

# CurrentPercent = np.array([a/100+0.01 for a in range(100)]) # 0.01 ~ 1.00
# StancePercent = 0.6
# SwingPercent = 1 - StancePercent

# x = np.zeros(len(CurrentPercent), dtype=float)
# y = np.zeros(len(CurrentPercent), dtype=float)

# for i in range(len(CurrentPercent)):
#      if (CurrentPercent[i] <= StancePercent):
#           x[i] = -(StepLength / 2) + (CurrentPercent[i] / StancePercent) * StepLength
#           y[i] =  DownAMP * np.sin(np.pi * CurrentPercent[i] / StancePercent) + StanceHeight
#      else:
#           x[i] = (StepLength / 2) - ((CurrentPercent[i] - StancePercent) / SwingPercent) * StepLength
#           y[i] = -UpperAMP * np.sin(np.pi * (CurrentPercent[i] - StancePercent) / SwingPercent) + StanceHeight

# plt.plot(x,y)
# plt.plot(x[99],y[99],x[79],y[79],x[59],y[59],x[29],y[29], marker='*')
# plt.title('SineTrajectory')
# plt.ylim(0.195, 0.125)
# plt.xlim(0.065, -0.065)
# plt.grid()
# plt.show()

In [42]:
# 1,2 for crawl

def SineTrajectory1(DownAMP=0.01, UpperAMP=0.08, StanceHeight=0.18, StepLength=0.15):
    
    StancePercent = 0.5
    SwingPercent = 1 - StancePercent

    t_iteration = 2
    t_real = time.time()
    t_cycle = t_real % 2
    CurrentPercent = t_cycle / t_iteration
        
    if (CurrentPercent <= StancePercent): # stance phase front foot
        x = -(StepLength / 2) + (CurrentPercent / StancePercent) * StepLength
        y =  DownAMP * np.sin(np.pi * CurrentPercent / StancePercent) + StanceHeight
    else: # swing phase front foot
        x = (StepLength / 2) - ((CurrentPercent - StancePercent) / SwingPercent) * StepLength
        y = -UpperAMP * np.sin(np.pi * (CurrentPercent - StancePercent) / SwingPercent) + StanceHeight

    return np.array([[x], [y]])

def SineTrajectory2(DownAMP=0.01, UpperAMP=0.08, StanceHeight=0.18, StepLength=0.15):
    
    StancePercent = 0.5
    SwingPercent = 1 - StancePercent

    t_iteration = 2
    t_real = time.time()
    t_cycle = t_real % 2
    CurrentPercent = t_cycle / t_iteration
    if (CurrentPercent <= StancePercent): # stance phase front foot
        x = (StepLength / 2)- (CurrentPercent / StancePercent) * StepLength
        y =  DownAMP * np.sin(np.pi * CurrentPercent / StancePercent) + StanceHeight
    else: # swing phase front foot
        x = -(StepLength / 2) + ((CurrentPercent - StancePercent) / SwingPercent) * StepLength
        y = -UpperAMP * np.sin(np.pi * (CurrentPercent - StancePercent) / SwingPercent) + StanceHeight
        
    return np.array([[x], [y]])

# 3, 4 for walk

def SineTrajectory3(DownAMP=0.01, UpperAMP=0.1, StanceHeight=0.18, StepLength=0.15):
    
    StancePercent = 0.5
    SwingPercent = 1 - StancePercent

    t_iteration = 2
    t_real = time.time()
    t_cycle = t_real % 2
    CurrentPercent = t_cycle / t_iteration
        
    if (CurrentPercent <= StancePercent): # stance phase front foot
        x = -(StepLength / 2) + (CurrentPercent / StancePercent) * StepLength
        y =  DownAMP * np.sin(np.pi * CurrentPercent / StancePercent) + StanceHeight
    else: # swing phase front foot
        x = (StepLength / 2) - ((CurrentPercent - StancePercent) / SwingPercent) * StepLength
        y = -UpperAMP * np.sin(np.pi * (CurrentPercent - StancePercent) / SwingPercent) + StanceHeight

    return np.array([[x], [y]])

def SineTrajectory4(DownAMP=0.01, UpperAMP=0.1, StanceHeight=0.18, StepLength=0.15):
    
    StancePercent = 0.5
    SwingPercent = 1 - StancePercent

    t_iteration = 2
    t_real = time.time()
    t_cycle = t_real % 2
    CurrentPercent = t_cycle / t_iteration
    if (CurrentPercent <= StancePercent): # stance phase front foot
        x = -(StepLength / 2) + (CurrentPercent / StancePercent) * StepLength
        y = -UpperAMP * np.sin(np.pi * (CurrentPercent / StancePercent)) + StanceHeight
    else: # swing phase front foot
        x = (StepLength / 2) - ((CurrentPercent - StancePercent) / SwingPercent) * StepLength
        y =  DownAMP * np.sin(np.pi * ((CurrentPercent - StancePercent) / SwingPercent)) + StanceHeight
                              
    return np.array([[x], [y]])

# Define virtual compliance

In [44]:
# stiffness coeff (currently can up to 100)
kx = 60
ky = 60
k  = np.array([[kx,0],[0,ky]])
# damping coeff
cx = 0.5
cy = 0.5
c  = np.array([[cx,0],[0,cy]])
# motor torque const
Kt = 0.0285

def cartesian_compliance_homePosition(theta_R, theta_L, vel_R, vel_L):
    
    J_trans  = -np.transpose(J_fast(theta_R, theta_L))  
    disp     = FK_fast(theta_R, theta_L) - FK_fast(theta_R_eq, theta_L_eq) 
    vel      = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)
    return currents 

def cartesian_compliance_sineTrajectory1(theta_R, theta_L, vel_R, vel_L):
    
    J_trans  = -np.transpose(J_fast(theta_R, theta_L))
    disp     = FK_fast(theta_R, theta_L) - SineTrajectory1()   
    vel      = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)
    return currents 

def cartesian_compliance_sineTrajectory2(theta_R, theta_L, vel_R, vel_L):
    
    J_trans  = -np.transpose(J_fast(theta_R, theta_L))
    disp     = FK_fast(theta_R, theta_L) - SineTrajectory2()   
    vel      = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)
    return currents 

def cartesian_compliance_sineTrajectory3(theta_R, theta_L, vel_R, vel_L):
    
    J_trans  = -np.transpose(J_fast(theta_R, theta_L))
    disp     = FK_fast(theta_R, theta_L) - SineTrajectory3()   
    vel      = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)
    return currents 

def cartesian_compliance_sineTrajectory4(theta_R, theta_L, vel_R, vel_L):
    
    J_trans  = -np.transpose(J_fast(theta_R, theta_L))
    disp     = FK_fast(theta_R, theta_L) - SineTrajectory4()   
    vel      = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)
    return currents 


# Control loop (each loop takes about 0.005s 200Hz)

In [50]:
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)

t_start = time.time()
duration = 10 #sec
count = 0

while (time.time() - t_start < duration):
    
    t0 = time.time()
    
    #current angle(rad) & velocity(rad/s)
    odrv1_theta_R = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
    odrv1_theta_L = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
    odrv2_theta_R = - ((m00.encoder.pos_estimate - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
    odrv2_theta_L = - ((m11.encoder.pos_estimate - m11_home_pos) / 8192 * 2 * np.pi)
    
    odrv1_vel_R = - ((m0.encoder.vel_estimate) / 8192 * 2 * np.pi)
    odrv1_vel_L = - ((m1.encoder.vel_estimate) / 8192 * 2 * np.pi)
    odrv2_vel_R = - ((m00.encoder.vel_estimate) / 8192 * 2 * np.pi)
    odrv2_vel_L = - ((m11.encoder.vel_estimate) / 8192 * 2 * np.pi)
    
    #get current
#     current = cartesian_compliance_homePosition(theta_R, theta_L, vel_R, vel_L)
    joint_damping_R = 0.5
    joint_damping_L = 0.5
    
    #get time right now
    #t_cyclenow = (time.time()) %2
    
    # run the commands based on time
    
    odrv1_current = cartesian_compliance_sineTrajectory1(odrv1_theta_R, odrv1_theta_L, odrv1_vel_R, odrv1_vel_L)
    odrv1_damping_current_R = np.array(joint_damping_R * odrv1_vel_R)
    odrv1_damping_current_L = np.array(joint_damping_L * odrv1_vel_L)
    odrv1_current_R = odrv1_current[0] - odrv1_damping_current_R
    odrv1_current_L = odrv1_current[1] - odrv1_damping_current_L
    
    odrv2_current = cartesian_compliance_sineTrajectory2(odrv2_theta_R, odrv2_theta_L, odrv2_vel_R, odrv2_vel_L)
    odrv2_damping_current_R = np.array(joint_damping_R * odrv2_vel_R)
    odrv2_damping_current_L = np.array(joint_damping_L * odrv2_vel_L)
    odrv2_current_R = odrv2_current[0] - odrv2_damping_current_R
    odrv2_current_L = odrv2_current[1] - odrv2_damping_current_L

    #make sure the current is not exceeds the limit
    odrv1_current_R = MaxCurrentProtection(odrv1_current_R)
    odrv1_current_L = MaxCurrentProtection(odrv1_current_L)
    odrv2_current_R = MaxCurrentProtection(odrv2_current_R)
    odrv2_current_L = MaxCurrentProtection(odrv2_current_L)
    
    #control the motors
    m0.controller.set_current_setpoint(-odrv1_current_R)
    m1.controller.set_current_setpoint(-odrv1_current_L)
    m00.controller.set_current_setpoint(-odrv2_current_R)
    m11.controller.set_current_setpoint(-odrv2_current_L)
    
    count += 1

    t1 = time.time()
    time_spent = t1 - t0
#     print(time_spent)
#     time.sleep(0.01 - time_spent)

print(count,'iterations in', duration, 'second')
print("Frequency:", count/duration, 'Hz')

m0.controller.set_current_setpoint(0)
m1.controller.set_current_setpoint(0)
m00.controller.set_current_setpoint(0)
m11.controller.set_current_setpoint(0)

ControllerPositionControl(odrv1)
ControllerPositionControl(odrv2)

ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)
    

1256 iterations in 10 second
Frequency: 125.6 Hz
Current Control Mode :3
Current Control Mode :3


In [48]:
IdleMode(odrv1)
IdleMode(odrv2)

In [ ]:
m0.controller.config.control_mode = 3
m1.controller.config.control_mode = 3
m00.controller.config.control_mode = 3
m11.controller.config.control_mode = 3
ClosedLoopControlMode(odrv1)
ClosedLoopControlMode(odrv2)